<h3> Featurizing text data with tfidf weighted word vectors</h3>

In [68]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
import spacy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [69]:
df = pd.read_csv('nlp_features_train.csv')

df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [70]:

# merge texts
questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy".
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics.

In [71]:
#!python -m spacy download en_core_web_lg
# !python -m spacy download en_core_web_sm
# !python -m spacy download en_core_web_md
#!python -m spacy download en

In [72]:
nlp = spacy.load('en_core_web_lg')

In [73]:
vecs1 = []

for qu1 in tqdm(list(df['question1'])):
    doc1 = nlp(qu1) # returns vector for each ques
 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])  #Create empty numpy array
    for word1 in doc1:
    # word2vec
        vec1 = word1.vector       #get a vector for that word using GLOVE
    # fetch idf score
        try:
            idf = word2tfidf[str(word1)]  #getting tfidf value for this word
        except:
            idf = 0       #if not available assign 0, like for stopwords
         # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
        
df['q1_feats_m'] = list(vecs1)

100%|██████████████████████████████████| 404287/404287 [22:02<00:00, 305.63it/s]


In [74]:
vecs2 = []
for qu2 in tqdm(list(df['question2'])):
    doc2 = nlp(qu2) 
    mean_vec2 = np.zeros([len(doc1), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df['q2_feats_m'] = list(vecs2)

100%|██████████████████████████████████| 404287/404287 [22:02<00:00, 305.65it/s]


In [75]:

#nlp_features_train.csv (NLP Features)
if os.path.isfile('nlp_features_train.csv'):
    dfnlp = pd.read_csv("nlp_features_train.csv",encoding='latin-1')
else:
    print("run previous PreProcessing notebook to get this file")


In [78]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)

df_q1 = pd.DataFrame(df.q1_feats_m.values.tolist(), index= df.index)
df_q2 = pd.DataFrame(df.q2_feats_m.values.tolist(), index= df.index)

In [81]:
# Questions 1 tfidf weighted word2vec
df_q1.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-3.267743,16.103498,3.012849,8.423555,17.968159,-5.226238,-4.697521,-1.628920,6.972993,142.904604,...,-17.579882,6.076222,1.081600,-8.423992,-1.251346,-11.003085,-12.594355,2.810165,2.048218,15.104115
1,9.671887,12.217163,18.287925,-2.048364,-15.900945,-1.837174,8.429692,-19.924049,-19.489964,11.860533,...,25.825724,0.189884,-10.353249,5.683133,29.353627,3.855112,-26.971075,12.650927,-7.857303,30.212451
2,1.144941,15.836792,-7.837756,-4.717320,-2.846363,9.611074,4.249176,-4.766542,6.501057,104.227409,...,-20.443719,2.344869,8.668823,-0.703825,15.721829,-2.807966,10.336125,-0.818313,-7.167827,18.883461
3,-4.813963,16.449937,-15.210961,0.903151,-2.341945,-0.391944,2.608291,-8.193090,2.670947,113.778389,...,-2.477880,-5.110100,-0.306457,-1.672598,9.200288,2.614452,4.975822,-1.056445,-3.024975,2.915154
4,-11.158058,19.905235,-4.730321,-6.290102,-21.067883,32.974064,-31.182229,1.147465,-19.044037,83.040520,...,-8.399529,-5.197304,19.033976,-39.863563,-10.685017,15.285213,-0.979483,-9.200051,-7.940075,23.474208


In [82]:
# Questions 2 tfidf weighted word2vec
df_q2.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.394202,13.775056,-0.465120,9.222053,13.640627,-4.681503,-5.315783,-0.138888,4.914662,133.245542,...,-17.747144,7.232094,1.578580,-7.447540,0.360983,-11.833308,-11.249363,1.852297,3.630329,11.942385
1,4.680233,9.616012,20.218212,-0.278613,-18.001907,-1.833364,5.027661,-27.433993,-20.899144,94.145610,...,22.829408,3.333261,-5.066854,6.977669,33.760693,6.037682,-27.214980,13.087034,-4.928176,33.398721
2,-16.654269,17.121892,-8.701906,-5.635221,-1.842484,4.291041,17.602291,-4.065899,2.474838,96.398623,...,-23.373241,-1.190748,11.905931,9.456080,10.872678,1.089150,6.161524,0.310342,-12.054247,26.727977
3,-1.068846,33.553180,24.733954,4.467639,3.404374,-9.552183,-11.942286,-2.739784,17.374891,140.514829,...,13.241865,-3.984300,-11.721567,12.137801,-9.194564,-21.156084,-7.106075,-19.471916,2.163286,2.875710
4,-5.422799,1.707931,1.799384,-4.020605,-3.632044,8.384369,-25.104362,7.496542,-2.744231,88.463352,...,-10.251953,-8.424718,-14.437992,-12.661327,-4.409586,12.534569,-11.637611,-16.320942,3.557839,-9.794110


In [83]:
print("Number of features in nlp dataframe :", df1.shape[1])

print("Number of features in question1 w2v  dataframe :", df_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df_q1.shape[1]+df_q2.shape[1])

Number of features in nlp dataframe : 28
Number of features in question1 w2v  dataframe : 300
Number of features in question2 w2v  dataframe : 300
Number of features in final dataframe  : 628


In [88]:
# storing the final features to csv file
if not os.path.isfile('final_features.csv'):
    df_q1['id']=df1['id']
    df_q2['id']=df1['id']
    df2  = df_q1.merge(df_q2, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv('final_features.csv')